# Chapter 11. Training Deep Neural Networks

Trong chapter 10, chúng ta đã giới thiệu  vể ANN và trained our first dêp neural networks. Nhưng chúng ta chỉ đang xửu lý vài hidden layer. Vậy sẽ ra sao nếu bạn sẽtackle 1 vấn đề lớn như, hát hiện 100 đối tượng trong 1 hình ảnh có độ phân giải cao. Train a deep DNN không phải **isn't walk in the park**. Đây sẽ là 1 vài ví dụ bạn sẽ có thể đi vào dưới note book này 

* đó chính là *vanishing gradients* *exploding gradients*
* không đu dữ liệu training cho 1 mạng lưới lớn , hoặc quá đát để mình gãn label 
* trianing có thể chậm 
* Overfitting the training set, hoặc thiếu training instances hoặc chúng quá là khó hiểu. 

Note book này sẽ going tất cả các vấn đề ở rteen và chúng ta sẽ có 1 vài kỹ thuật để giải chúng. CHúng ta sẽ bắt đàu exporing với *vanishing and exploding* gradinet. Tiếp đến chúng ta sẽ đi dến *transfer learning* và *unsupervised pretraining* nó có thể hỗ trợ chúng ta tiếp cận những vấn đề lớn ngay cả khi chúng ta có ít label. Rồi chúng ta sẽ thảo luận các tối ưng để tăng lượng speed up khi train 1 với 1 mô hình lớn. Và chúng ta sẽ đi qua 1 vài những technich regularzation cho tập lớn mạng lưới neraul  

## The Vanishing/Exploding Gradients Problems

Như chúng ta đã thảo luận trong **Chapter 10**, the backpropagation algo hoạt động bằng cách đi từ output layer to input layer, lan truyền thẻ **error gradient** dọc theo dường đi.Khi thuật toán đã tính gradient của hàm mất mát liên quan tới từng tham số trong network , nó sử dụng gradient để update các parameter với a Gradient Descen step.

Thật không may, gradient thường lấy giá trị nhỏ và và nhỏ hơn khi thuật toán đi tới xuống các layer dưới. Như một kết quả, the Gradeint Descen update cập nhật các connection weight ở dưới hầy như như không thay đổi, và sẽ ko bao giờ đạt được good solution. Chúng tôi gọi là **vanishing gradients** problem. Trong 1 vài trường hợp nó có thể xảy ra, gradient có thể càng lớn và lớn hơn cho đến khi các lớp nhận được cập nhật trọng lượng cực lớn và thuật toán đó là phân kỳ. Đó là **exploding gradient**.Nói một cách tổng quát hơn, các mạng nơ-ron sâu có độ dốc không ổn định; các lớp khác nhau có thể học ở nhiều tốc độ khác nhau. Nói ngắn gọn, họ đã chỉ ra rằng với this activation và init scheme the variance của outputs của mỗi layer lớn hơn rất nhiều variance của inputs. Vào trong cái network, phương sai sẽ tiếp tục tăng sau khi mỗi layer cho đến activation function ở các lớp trên cùng. Sự saturation thực sự tồi tệ bởi thực sự nó là hàm logistic có trung bình 0.5 chứ ko 0 .Do đó,khi quá trình lan truyền khi backgrogation truyền qua nó hầu như không có  gradient truyền ngược lại the network , và những cái ít graindent tồn tại tiếp tục bị lõng như backproapgation tiến xuống các lớp trên cùn, và điều đó là thực sự k có còn gì cho các lớp dưới  

### Glorot and He Initialization
Trong bài báo của, Glorot and Bengio đã đề xuất 1 cách đáng kể vấn đề Gradient không ổn đinh. Họ đề xuất 1 cáchh giảm đáng kể vấn đề gradient không ổn đinh. Họ chỉ ra rằng chúng ta cần tín hiệu lưu thông đúng theo cả 2 hướng, theo hướng đằng trướng khi chúng ta predictions, và theo hướng ngược lại khi backprogating gradient. Chúng tôi không muốn tín hiệu bị tắt cũng như chúng tôi không muốn nó bão hoà hay bùnh nổ. Để tín hiệu đi đúng hướng các tác giả lập luận rằng chúng ta cần variance của outputs của outputs của mỗi layes bằng variance của input. và chúng ta cần gradient và chúng ta cần các gradient có phương sai bằng nhau trước và sau khi chảy qua một lớp theo hướng ngược lại. Nó thực sự không Thực sự không thể đảm bảo cả hai trừ khi lớp có số lượng đầu vào và tế bào thần kinh bằng nhau . nhưng Glorot và Bengio đã đề xuất một thỏa hiệp tốt đã được chứng minh là hoạt động rất tốt trong thực tế: trọng số kết nối của mỗi lớp phải được khởi tạo ngẫu nhiên như công thức dưới đây 

$fan_{avg} = (fan_{in} + fan_{out})/2$

Phân phối chuẩn với MAEN = 0 và 
variance $\sigma^{2}$ = $\frac{1}{fan_{avg}}$

default, keras sử dụng Glorot init với phân phối đồng nhất. khi tạo 1 layer, chúng ta có thêr thay đổi this to He init bởi setting 

```
keral_initializer="he_uniform" or 
keral_inittializer="he_normal" 

keras.layers.Dense(10,activation="relu",kernel_inititalizer="he_norlmal"
```
Nếu bạn muốn He Init với 1 uni form distribution nhưng dựa trên on $fan_{avg}$ thay vì $fan_{in}$ bạn có thể sửu dụng VarianceScaling inittialier like this:

```
he_avg_init = keras.initializers.VarianceScaling(scale=2,mode="fan_avg",
distribution='uniform'
keras.layers.Dense(10,activation="sigmoid",kernel_inittializer=he_avg_init)
```

### Nonsaturating Activation Functions
Một trong những hiểu biết sâu sắc trong 2010 là vấn đề với **unstatble gradient** là 1 phần do lựa chọn sai activation function. Vì hầu hết mọi người đều nghĩ rằng mặc định `sigmoid` activation trong bioligical neurasl. Nó là sự lựa chọn oke nhất tại thời điểm đó. nhưng cái khác hoạt động tốt hơn trong deep neural networkds trong thực tế đó là ReLu activation function. chủ yếu vì nó không **saturate** cho giá trị dương và nó tính nhanh.

Thật không may, ReLU activation function không thực sự tốt. Nó mắc phải 1 vấn đề được biết là **dying ReLUs** trong suốt quá trình huấn luyện , 1 vài tế bào sẽ chết nghĩa là chúng sẽ dừng output không khác gì 0. Trong 1 vài trường hợp, bạn có thể thấy một nửa netwwork's neurosn sẽ chết nếu chúng ta sử dụng 1 lượng lớn large learning rate. 1 Neuron **dies** khi trọng số của nó được điều chỉnh theo 1 cách tổng các weight của input là số âm cho tất cả các thành phần của tập huấn luyện. Khi điều này xảy ra, nó sẽ giữ input là 0 và Gradient Descen không phát huy tác dụng nữa.

Để sử lý vấn đề này, bạn có thể sử dụng 1 biến thể của ReLu function ví dụ như là *leaky* ReLU. Func này được định nghĩa 
$LeakyReLU_{\alpha}(z) = max(\alpha z,z)$

Hyperparameter $\alpha$ được định là bao nhiêu cái mà function này nó bị *leak*. nó là 1 small độ dốc mà thiếu ReLus để nó không chết. A paper 2015 đã so sánh ReLU activation , và 1 các biết thể của nó và đã kết luận rằng **leaky ReLu** luông luôn có output tốt hơn.
*  **RReLu**(xử lý tốt  và hoạt đông như là 1 reguarizer) 
*  **PReLu** xử lý cực tốt hơn ReLU trên image dataset 
Còn 1 cái nhưng k kém phần quan trọng. **ELU**  (exponential linear unit) output xử lý tốt hơn perfornane các biến thể của ReLU



$$ELU_{\alpha}(z) = \left\{ \begin{array}{rcl}
{\alpha}(exp(z)-1\ & \mbox{if}
& z<0 \\ z & \mbox{for} & z>= 0
\end{array}\right.$$

The ELU activation Function nhìn có thể giống nhu ReLU nhưng có 1 vài điểm khác biệt 
* Nó nhận các giá trị âm khi z <0, cho phép đơn vị có đầu ra trung bình gần với 0 và giúp giảm bớt vấn đề về độ dốc biến mất. Hyperparameter α xác định giá trị mà hàm ELU tiếp cận khi z là một số âm lớn. Nó thường được đặt thành 1, nhưng bạn có thể điều chỉnh nó giống như bất kỳ siêu thông số nào khác  
* Tránh z < 0  dead neurons 

Điểm hạn chế của ELU là nó chậm tính toán hơn ReLUTốc độ hội tụ nhanh hơn của nó trong quá trình đào tạo bù đắp cho tính toán chậm đó, tuy nhiên, tại thời điểm thử nghiệm, mạng ELU sẽ chậm hơn mạng ReLU 

Trong 2017 paper, đã giới thiệu SELU activation, nó là 1 biến thể được chia tỷ lệ của ELU.  SELU activation function network sẽ tự chuẩn hoá outpur sẽ hướng tới mean =0 std = 1 xuyên xuốt quá trình traning set , nhưng nó cvài điều kiện sau đây: 

* Input phải standardized 
* Mỗi hidden latet phải init bằng kernel_init ="lecun_normal" 
* Architecture phải sequental 
* tất cả layer là dense kết nối full, nhưng 1 vài nghiên cứu nói rằng SELI có thể improve performance trong CNN. 

Để sử dụng leaky ReLU activation, create **Leaky ReLu** layet and thêm nó vào model 
```
model = keras.modesl.Sequantial([
  keras.layers.Dense(10,kernel_initlizer="he_normal",
  keras.layers.LeakyReLU(alpha=0.2)
])

```
Với PReLU, replace cái LeakyRelu với PReLU()

Với thằng SELU set activation ="selu" và set cái init là lecun_normal" cái này bắt buộc nhac 

### Batch Normalization

Mặc dù sử dụng He Init vcùng với ELU hoặc với bất kỳ biết thể nào của RELU , nó có là giảm đáng kể sử nguy hiểm của vanisshing/ exploding gradients tại bắt đầu huấn luyện, nhưng nó k đảm bảo rằng chúng không thể quay lại xuyên suốt trong quá trình đào tạo 

Trong 1 tờ 2015 paper Sergey Ioffe and Chirstian Szegedy đã chứng minh rằng 1 kỹ thuật được gọi là *Batch Normalization* (BN) sẽ xử lý vấn đề ở trên. Cái kỹ thuật này bao gồm thêm những operation trong model ngay trước hoặc sau mỗi activation function cho mỗi hidden layer. Operation này chỉ đơn giản là định tâm và chuẩn hóa từng đầu vào, sau đó chia tỷ lệ và thay đổi kết quả bằng cách sử dụng hai vectơ tham số mới trên mỗi lớp , 1 cho scaling và cái khác cho shifting. Mặt khác các operation hướng tới model học các tối ưu scale và mean của mỗi layers in put. Trong nhiều trường hợp nếu chúng ta thêm 1 BN layers như là first layer trong network thì chúng ta **không cần phải Standarscaler**

#### Implementing Batch Normalization with KEras 

In [1]:
#import library 
import tensorflow as tf
from tensorflow import keras


In [3]:
#import data set 
fashion_mnist = keras.datasets.fashion_mnist
(X_train,y_train),(X_test,y_test)= fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [4]:
X_train.dtype

dtype('uint8')

In [5]:
X_train.shape

(60000, 28, 28)

In [8]:
 
# set seed 
tf.random.set_seed(42)
# Creating the model with Sequential API
model_batch_normalization = keras.models.Sequential([
          keras.layers.Flatten(input_shape=[28,28]),
          keras.layers.BatchNormalization(),
          keras.layers.Dense(300,activation='elu',kernel_initializer='he_normal'),
          keras.layers.BatchNormalization(),
          keras.layers.Dense(100,activation="elu",kernel_initializer='he_normal'),
          keras.layers.Dense(10,activation='softmax')
])
# compiling the model
model_batch_normalization.compile(
    loss= keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics =['accuracy']
)
# Fiting the model
model_batch_normalization.fit(X_train,y_train,epochs=10)

Epoch 1/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.4713 - accuracy: 0.8313
Epoch 2/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.3553 - accuracy: 0.8687
Epoch 3/10
1875/1875 [==============================] - 12s 7ms/step - loss: 0.3163 - accuracy: 0.8827
Epoch 4/10
1875/1875 [==============================] - 12s 7ms/step - loss: 0.2901 - accuracy: 0.8912
Epoch 5/10
1875/1875 [==============================] - 12s 7ms/step - loss: 0.2685 - accuracy: 0.8989
Epoch 6/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2500 - accuracy: 0.9063
Epoch 7/10
1875/1875 [==============================] - 12s 7ms/step - loss: 0.2360 - accuracy: 0.9099
Epoch 8/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2252 - accuracy: 0.9138
Epoch 9/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2068 - accuracy: 0.9214
Epoch 10/10
1875/1875 [==============================] - 11s 6ms/step - l

Các tác giả của bài báo về BN đã lập luận ủng hộ việc thêm các lớp BN trước các **activation function**, thay vì sau (như trên vừa làm). Có một số cuộc tranh luận về điều này, vì điều nào tốt hơn dường như phụ thuộc vào **my problem** cũng có thể thử nghiệm điều này để xem tùy chọn nào hoạt động tốt nhất trên dataset. Để thêm các lớp BN trước các "activation functon"t, ta phải xóa "activation functon"t khỏi các Hidden layer và thêm chúng dưới dạng các lớp riêng biệt sau các lớp BN.Thêm nữa, vì Batch Normalization layer bao gồm thêm 1 ofset parameter trong mỗi input, chúng ta có thể remove bias từ các layer trước

In [10]:
# set seed 
tf.random.set_seed(42)
# Create the model with Sequential API 
model_batch_normalization_author = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300,kernel_initializer='he_normal',use_bias=False),  
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),
   
    keras.layers.Dense(300,kernel_initializer='he_normal',use_bias=False),    
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(10,activation="softmax")                                        
])
# Compile the model 
model_batch_normalization_author.compile(
    loss= keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics =['accuracy']
)
# Fit the model 
model_batch_normalization_author.fit(X_train,y_train,epochs=10)

Epoch 1/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.4895 - accuracy: 0.8250
Epoch 2/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.3911 - accuracy: 0.8581
Epoch 3/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.3534 - accuracy: 0.8701
Epoch 4/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.3297 - accuracy: 0.8783
Epoch 5/10
1875/1875 [==============================] - 16s 9ms/step - loss: 0.3078 - accuracy: 0.8852
Epoch 6/10
1875/1875 [==============================] - 16s 9ms/step - loss: 0.2917 - accuracy: 0.8917
Epoch 7/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2786 - accuracy: 0.8949
Epoch 8/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2674 - accuracy: 0.8997
Epoch 9/10
1875/1875 [==============================] - 18s 10ms/step - loss: 0.2522 - accuracy: 0.9059
Epoch 10/10
1875/1875 [==============================] - 17s 9ms/step - 